In [30]:
import finnhub
import os
from pydantic import BaseModel
from dotenv import load_dotenv
from typing import Optional
import time

load_dotenv()

class StockPriceResponse(BaseModel):
    current_price: float
    previous_close: float
    change: Optional[float] = None
    change_percent: Optional[float] = None
    high: Optional[float] = None
    low: Optional[float] = None
    open: Optional[float] = None

def get_stock_price(symbol: str) -> StockPriceResponse:
    """
    Get stock price using Finnhub API
    
    Args:
        symbol: Stock ticker symbol (e.g., 'AAPL', 'GOOGL')
    
    Returns:
        StockPriceResponse with current and previous close prices
    
    Raises:
        Exception: If API call fails or returns invalid data
    """
    # Initialize Finnhub client with API key from .env
    api_key = os.getenv('FINHUB_API_KEY')
    if not api_key:
        raise ValueError("FINHUB_API_KEY not found in .env file")
    
    finnhub_client = finnhub.Client(api_key=api_key)
    
    try:
        # Get quote data for the symbol
        quote = finnhub_client.quote(symbol.upper())
        
        # Check if we got valid data
        if not quote or quote.get('c') is None:
            raise ValueError(f"No data available for symbol: {symbol}")
        
        # Extract the data
        current_price = quote['c']  # Current price
        previous_close = quote['pc']  # Previous close
        
        # Create response with additional data if available
        return StockPriceResponse(
            current_price=current_price,
            previous_close=previous_close,
            change=quote.get('d'),  # Change ($)
            change_percent=quote.get('dp'),  # Change (%)
            high=quote.get('h'),  # Day high
            low=quote.get('l'),  # Day low
            open=quote.get('o')  # Day open
        )
        
    except Exception as e:
        raise Exception(f"Failed to fetch stock price for {symbol}: {str(e)}")


In [31]:
# RIVN - Rivian Automotive, Inc.
# LCID - Lucid Group, Inc.
# TSLA - Tesla, Inc.

get_stock_price("LCID") 

StockPriceResponse(current_price=18.41, previous_close=16.16, change=2.25, change_percent=13.9233, high=18.735, low=16.3, open=16.34)

In [32]:
# Send SMS with Twilio
from twilio.rest import Client


account_sid = os.getenv("ACCOUNT_SID")
auth_token = os.getenv("AUTH_TOKEN")
to_phone_number = os.getenv("TO_PHONE_NUMBER")
from_phone_number = os.getenv("FROM_PHONE_NUMBER")
client = Client(account_sid, auth_token)

def send_sms( body: str) -> str:
    """
    Send SMS using Twilio API
    
    Args:
        to: Recipient phone number (in E.164 format, e.g., '+1234567890')
        body: Message body
        from_: Sender phone number (Twilio number)
    
    Returns:
        Message SID if sent successfully
    
    Raises:
        Exception: If sending fails
    """
    try:
        message = client.messages.create(
            body=body,
            from_=from_phone_number,
            to=to_phone_number
        )
        return message.sid
    except Exception as e:
        raise Exception(f"Failed to send SMS to {to_phone_number}: {str(e)}")


In [5]:
# Test sending SMS
send_sms("Hello from Twilio and Python!")

'SM820823d91ef250653ce32551b941bd52'

In [33]:
import json

def get_tracker_list():
    with open("stocks_lists.json", "r") as f:
        tracker_list = json.load(f)

    return tracker_list

In [34]:
get_tracker_list()

['RIVN', 'LCID', 'TSLA']

In [35]:
from agents import Agent, FileSearchTool, Runner, WebSearchTool, function_tool

@function_tool
async def fetch_stock_price(symbol: str) -> StockPriceResponse:
    return get_stock_price(symbol)


@function_tool
async def send_stock_sms( body: str) -> str:
    return send_sms(body)

@function_tool
async def get_tracker_list_tool() -> dict:
    return get_tracker_list()

In [44]:
from agents import Agent

stock_research_agent = Agent(
  name="Stock Research Agent",
  instructions="""
  You are a Stock Market Researcher. Your job is to research the current news around a specific stock and determine what may have caused recent price movements in the past 24 hours specifically. You have access to be able to retreive the stock price and research via the internet using your tools.

  Use the get_tracker_list tool to retrieve the list of stocks being tracked. Make sure you have an explanation for the movement in the stock price.

  You should use the fetch_stock_price tool to check current stock price, previous close, and other relevant data. Use the WebSearchTool to find recent news articles or information about the stock.

  Output: Your final_output should be a the price, an short analysis of the price change and short summary of your findings, send the summary of each stock via SMS in a unique message to the user using the send_stock_sms tool as the body parameter.
  """,
  tools=[
    fetch_stock_price,
    WebSearchTool(),
    send_stock_sms,
    get_tracker_list_tool
  ],
  model="gpt-4.1"
)

In [45]:
from agents import Runner

async def main():
    result = await Runner.run(stock_research_agent, "Research the stocks in the tracker list and send an SMS with your findings")
    print(result.final_output)

In [46]:

import asyncio

if __name__ == "__main__":
    await main()

Here are the SMS summaries for the stocks in your tracker list:

1. **RIVN:** Closed at $14.21 (+3.72%). Recent gains are linked to a new East Coast HQ in Atlanta, opening orders for electric commercial vans, and a software joint venture with Volkswagen—boosting investor confidence.

2. **LCID:** Closed at $18.41 (+13.92%). The sharp increase comes after a 1-for-10 reverse stock split, analyst upgrades, and a major $1.5B investment from the Saudi Public Investment Fund affiliate—driving positive momentum.

3. **TSLA:** Closed at $350.84 (+3.64%). The jump is attributed to a proposed $1T incentive package for CEO Elon Musk and positive movement on Tesla’s new UK energy license—sending the stock higher.

Each summary has been sent to you via SMS with the latest pricing and reasons for the price changes in the past 24 hours.
